# IN PROGRESS

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
###Set notebook preferences##
#Read in some libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import Image

#Set notebook preferences - pandas
pd.set_option("display.max_columns", 101)

#Set path to modeling folder
model_viz = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Spain Hourly Energy Demand and Weather\Visualizations\Modeling'

#Set path to data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Spain Hourly Energy Demand and Weather\Data\02_Cleaned_Data'

# 1. Introduction

Airbnb is easily one of the most successful start-ups in recent memory. Airbnbn's business model has been a disruptive force in the short-term rental market and was valued at \\$31 billion prior to the Corona Virus epidemic ([source](https://www.reuters.com/article/us-airbnb-debt/airbnbs-new-1-billion-investment-comes-at-lower-valuation-sources-idUSKBN21P3IM#:~:text=(Reuters)%20%2D%20Airbnb%20Inc's%20new,the%20matter%20said%20on%20Tuesday.&text=Airbnb%20was%20valued%20at%20%2431,recent%20private%20fund%2Draising%20round.)).
Howeevern in a recent artivle in April(just after massive lockdowns swept the world), the industry titans valuation has plummeted to nearly half ([source](https://skift.com/2020/04/08/airbnbs-valuation-almost-halved-to-18-billion/)).

In this analysis, I will be exploring the Airbnb rental market in San Francisco from 7/1/2018 through 6/7/2020. I intend to develop a better understanding about the impact COVID-19 is having on the SF market.

## 1.1 Intent

## 1.2 Objectives

- Conduct an analysis of the impact COVID-19 has had on daily gross rental income in the San Francisco area since the lockdown began on 3/19/2020. Forecast 2020 Q2, Q3, and Q4 losses if lockdown conditions remain the same for the rest of the year.
- Create a per night rental price predictor for Airbnb properties in San Francisco. Extract out the most pertinent features that predict prices best. 

## 1.3 Impact

I hope to use the information gained from the analysis to provide the following:
- Proojectug losses for the year will help company leadership manage expecations for profist and table/priotitze certain projects over others.
- Provide first-time hosts a guide of features/services to consider including in their Airbnb listing to increase the liklihood of interests from tourists as well as justify a higher rental price. Developing a good price predictor will also provide renters a range of acceptavble prices to list their property for(as compared to similar properties in SF)

# 2. Table of Contents

1. **Introduction**<br>
   1.1 Intent<br>
   1.2 Objectives<br>
   1.3 Impact<br>
2. **Table of Contents**<br>
3. **Data Overview**<br>
    3.1 About the Data<br>
    3.2 Preview Data<br>
    3.3 Data Overview - Metrics<br>
4. **Impact of COVID 19 on Gross Rental Revenue**<br>
    4.1 Methodology<br>
    4.2 Results<br>
    4.3 Decision Recommendations<br>
5. **Airbnb Rental Prices in San Francisco**<br>
    5.1 Methodology<br>
    5.2 Original TSO Forecast<br>
    5.3 Mulitvariate XGBRegressor Forecast<br>
    5.4 Decision Recommendations<br>
6. **Conclusion**<br>
    6.1 Takeaways<br>
    6.2 Future Research<br>

# 3. Data Overview

## 3.1 About the Data

In [ ]:
## 3.2 Preview Data

In [ ]:
## 3.3 Data Overview - Metrics